<a href="https://colab.research.google.com/github/kevin866/kevin866/blob/main/ebay_ml_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#import gzip
#import shutil
#with gzip.open('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv.gz', 'rb') as f_in:
 #   with open('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv', 'wb') as f_out:
 #       shutil.copyfileobj(f_in, f_out)
#unzip the gz file and copy it to the directory

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers

In [ ]:
!pip install -U spacy -q

In [ ]:
!python -m venv .env
!.env\Scripts\activate
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda113,transformers,lookups]'
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
!python -m spacy info

In [ ]:
from spacy.tokens import DocBin
db = DocBin()

In [ ]:
from tqdm import tqdm

In [ ]:
train_path = '/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Train_Tagged_Titles.tsv'

In [ ]:
train_df = pd.read_csv(train_path, sep='\t', engine='python',quoting=3)

In [ ]:
Listing_path = '/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/Listing_Titles.tsv'

In [ ]:
Listing_df = pd.read_csv(Listing_path, sep='\\\t', engine='python')

In [ ]:
tags = pd.DataFrame(train_df['Tag'].unique()).dropna()

In [ ]:
tags = np.ravel(np.array(tags).tolist())

In [ ]:
train = Listing_df.iloc[:5000]['Title']

In [ ]:
def merge_tagged_identities(train_df):
  train_df = pd.DataFrame(train_df)
  drop = []
  processed = train_df.copy(deep=True)
  count = 0
  for i, j in train_df.iterrows():
    if str(j[3]) == 'nan':
      count += 1
      drop.append(i)
    elif count > 0:
      processed.iat[i-count-1,2]=" ".join([processed.iloc[i-j][2] for j in range(1,count+2)][::-1])
      count = 0
  processed=processed[processed['Tag'].notna()]
  return processed

In [ ]:
processed = processed.reset_index(drop = True)

In [ ]:
processed.head()

In [ ]:
def extract_entites(train_df):
  processed = merge_tagged_identities(train_df)
  all_ent = []
  pos = 0
  ent = {'entites':[]}
  for i, j in processed.iterrows():
    if pos != 0:
      pos = pos + 1
    ent['entites'].append((pos, pos+len(j[2]), j[3]))
    pos = pos+len(j[2])
    if i+1 < len(processed):
      if processed.iloc[i+1]["Record Number"]>j[0]:
        pos = 0 
        all_ent.append(ent)
        ent = {'entites':[]}
  all_ent.append(ent)
  return all_ent

In [ ]:
def merge_entites_w_data(train, train_df):
  all_ent = extract_entites(train_df)
  train = pd.DataFrame(train)
  all_ent = pd.DataFrame(all_ent)
  n_train = train.join(all_ent)
  n_train['entites'] = n_train['entites'].apply(lambda x:{'entities': x})
  exam = n_train.apply(lambda x : (x[0],x[1]), axis = 1)
  exam = np.array(exam).tolist()
  return exam

In [ ]:
exam = merge_entites_w_data(train, train_df)
training = exam[:4000]
validation = exam[4000:]

In [ ]:
import json
def convert_to_docbin(data, tags, filename):
  data={"annotations":data,"classes": tags.tolist()} 
  data = json.dumps(data)
  loaded_data = json.loads(data)
  type(loaded_data) 
  for text, annot in tqdm(loaded_data["annotations"]):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
      span = doc.char_span(start, end, label = label, alignment_mode="contract")
      if span is None:
        print(label)
        print("skipping entity")
      else:
        ents.append(span)
    doc.ents = ents
    db.add(doc)
  db.to_disk("./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/{}".format(filename))




In [ ]:
convert_to_docbin(training, tags, 'training.spacy')
convert_to_docbin(validation, tags, 'validation.spacy')


In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_4_1 --paths.train ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/training.spacy --paths.dev ./drive/MyDrive/eBay_ML_Challenge_Dataset_2022/validation.spacy

In [ ]:
import spacy
nlp2 = spacy.load('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/output_4_1/model-best')

In [ ]:
label = processed[['Record Number','Token', 'Tag']].copy()
quiz_data = Listing_df[:5000]

In [ ]:
def prediction(quiz_data):
  column_names = ['Record number','Aspect Name', 'Aspect Value']
  predicted = pd.DataFrame(columns = column_names)
  count = 0
  for i, j in quiz_data.iterrows():
    doc2 = nlp2(j[1])
    for ent in doc2.ents:
      predicted.loc[count] = [j[0], ent.label_, ent.text] 
      count+=1
  return predicted

In [ ]:
def drop_tag(predicted):
  predicted = predicted[predicted['Aspect Name'] != 'Obscure']
  predicted = predicted[predicted['Aspect Name'] != 'No Tag']
  predicted=predicted.set_index('Record number', drop = True)
  return predicted

In [ ]:
predicted = prediction(quiz_data)

In [ ]:
label.columns = ["Record Number","Aspect Name","Aspect Value"]

In [ ]:
len(predicted)

In [ ]:
predicted.to_csv('/content/drive/MyDrive/eBay_ML_Challenge_Dataset_2022/quiz.tsv', header=None, sep="\t")